<a href="https://colab.research.google.com/github/Aserkanturunc07/My-projects/blob/main/Reverse_DCF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reverse DCF steps:
- Get current price data
- get free cash flow TTM of last 5 years
- get average P/E and P/FCF ratios for the last 10 years
- calculate the implied growth rate for the next 10 years (after 10 years just get the 5 year average P/FCF to calculate price)


To do's:

In [26]:
import yfinance as yf
import numpy as np
import pandas as pd

# Function to get current share price
def get_current_price(ticker):
    stock = yf.Ticker(ticker)
    price = stock.history(period="1d")['Close'].iloc[-1]
    return price

# Function to get number of shares outstanding
def get_shares_outstanding(ticker):
    stock = yf.Ticker(ticker)
    shares = stock.info['sharesOutstanding']
    return shares

# Function to get historic price data for the last 10 years
def get_historic_price(ticker):
    stock = yf.Ticker(ticker)
    hist = stock.history(period="10y")
    return hist['Close']

# Function to get TTM Free Cash Flow for the last 10 years
def get_fcf(ticker):
    stock = yf.Ticker(ticker)
    cashflow = stock.cashflow
    fcf = cashflow.loc['Free Cash Flow']
    return fcf

# Function to get net income (earnings)
def get_net_income(ticker):
    stock = yf.Ticker(ticker)
    earnings = stock.financials.loc['Net Income']
    return earnings

# Function to calculate the implied growth rate using DCF
def implied_growth_rate(price, fcf_per_share, r, N, terminal_value, shares_outstanding, buyback_rate, use_multiple=True):
    def firm_value(g):
        shares = shares_outstanding
        FCFs = []
        for t in range(1, N+1):
            shares *= (1 - buyback_rate)
            FCF = (fcf_per_share * (1 + g)**t) * shares
            FCFs.append(FCF)
        PV_FCFs = [FCF / (1 + r)**t for FCF, t in zip(FCFs, range(1, N+1))]
        if use_multiple:
            term_value = FCFs[-1] * terminal_value
        else:
            term_value = FCFs[-1] * (1 + terminal_value) / (r - terminal_value)
        PV_terminal_value = term_value / (1 + r)**N
        return sum(PV_FCFs) + PV_terminal_value

    g_low = 0.00
    g_high = 0.20  # Adjusted high value to 20% to allow for a reasonable range
    tolerance = 1e-6

    while g_high - g_low > tolerance:
        g_mid = (g_high + g_low) / 2
        value = firm_value(g_mid)
        if value < price:
            g_low = g_mid
        else:
            g_high = g_mid

    return g_mid

# Example usage
ticker = "NKE"

# Get current price
current_price = get_current_price(ticker)
print(f"Current Price: {current_price}")

# Get number of shares outstanding
shares_outstanding = 1508000000  # Manually set for testing, typically get from API
print(f"Shares Outstanding: {shares_outstanding}")

# Get historical prices for the last 10 years
historical_prices = get_historic_price(ticker)
print(historical_prices)

# Get TTM Free Cash Flow for the last 10 years
fcf = get_fcf(ticker)
print(f"Free Cash Flow:\n{fcf}")

# Get net income (earnings)
earnings = get_net_income(ticker)
print(f"Net Income:\n{earnings}")

# Calculate FCF per share
latest_fcf = fcf.dropna().iloc[0]  # Assuming the latest FCF value, dropping NaN values
fcf_per_share = latest_fcf / shares_outstanding
print(f"FCF per Share: {fcf_per_share}")

# Set parameters
r = 0.10  # Discount rate
N = 10  # Forecast period in years
multiple = 20  # FCF multiple after the 10th period
gT = 0.03  # Terminal growth rate after the 10th period
buyback_rate = 0.01  # Annual share buyback rate

# Calculate implied growth rates
implied_g_multiple = implied_growth_rate(current_price, fcf_per_share, r, N, multiple, shares_outstanding, buyback_rate, use_multiple=True)
print(f"The market's implied yearly growth rate using a P/FCF multiple of 20 is approximately {implied_g_multiple:.4f}")

implied_g_terminal = implied_growth_rate(current_price, fcf_per_share, r, N, gT, shares_outstanding, buyback_rate, use_multiple=False)
print(f"The market's implied yearly growth rate using a terminal growth rate of 3% is approximately {implied_g_terminal:.4f}")


Current Price: 72.55999755859375
Shares Outstanding: 1508000000
Date
2014-07-28 00:00:00-04:00    35.069794
2014-07-29 00:00:00-04:00    34.935596
2014-07-30 00:00:00-04:00    35.611057
2014-07-31 00:00:00-04:00    34.501694
2014-08-01 00:00:00-04:00    34.345131
                               ...    
2024-07-22 00:00:00-04:00    74.860001
2024-07-23 00:00:00-04:00    73.400002
2024-07-24 00:00:00-04:00    71.089996
2024-07-25 00:00:00-04:00    71.410004
2024-07-26 00:00:00-04:00    72.559998
Name: Close, Length: 2517, dtype: float64
Free Cash Flow:
2024-05-31    6617000000.0
2023-05-31    4872000000.0
2022-05-31    4430000000.0
2021-05-31    5962000000.0
2020-05-31             NaN
Name: Free Cash Flow, dtype: object
Net Income:
2024-05-31    5700000000.0
2023-05-31    5070000000.0
2022-05-31    6046000000.0
2021-05-31    5727000000.0
Name: Net Income, dtype: object
FCF per Share: 4.387931034482759
The market's implied yearly growth rate using a P/FCF multiple of 20 is approximately 0.

In [22]:
print(shares_outstanding)

1508000000
